In [505]:

import numpy as np
import matplotlib.pyplot as plt
from scipy.special import erf
from scipy.interpolate import interp1d
from scipy.optimize import curve_fit
import matplotlib.lines as mlines
import ipywidgets as widgets
from ipywidgets import Layout
from math import exp
from lmfit import Model, Parameters
import sys
e=2.718281828

In [ ]:
import numpy as np

In [507]:
filename="22BPNODIPS chloroform 600 200mW PTA fixed.csv"#22BPNODIPS chloroform 600 200mW PTA fixed.csv
#P-1-3A-P-ex600nm-50uW-fs.csv
rawdata = open(filename)

#step 1: split the data from the info at the end
#making it into new files: optional, can stick them into variables instead
fileinfo=filename[0:-4]+"-info"+".csv"
fileinfofile= open(fileinfo, "w") #'w' makes it write only so deletes prev version of it
justdata=filename[0:-4]+"-data"+".csv"
justdatafile=open(justdata, "w")

for line in rawdata:
    if len(line)<100: #if it is not data but text
        fileinfofile.write(line) 
    #step 2: remove the NaNs at the begining
    else: 
        nan=False
        for char in line:
            if char == 'N':
                nan=True
                break
        if nan:
            continue
        else:
            justdatafile.write(line)
fileinfofile.close() #its good practice to close files :)
rawdata.close()
justdatafile.close()
print((open(fileinfo)).read())


Date: March 16, 2015
Sample: NODIPS-22BP-Chloroform
Solvent: THF
Pump energy (uJ): 200 uW
Pump wavelength (nm): 600
Cuvette length (mm): 2
Comments: Time Zero: 505.800 ps
Averaging time: 2.0 s
Number of scans: 2
Measurement time: 00:37:49

Time units: ps
Z axis title: dA


In [271]:
dataarray = (np.genfromtxt(justdata, delimiter=",",  filling_values=0)).T
print(np.shape(dataarray))
X=dataarray[0,1:].T #works
Y=dataarray[1:,0]
Z=dataarray[1:,1:]
print(np.shape(X))
print(np.shape(Y))
print(np.shape(Z))

(456, 454)
(453,)
(455,)
(455, 453)


In [272]:
#step 3: remove the noise at the begining and end
#read the begining, check if its noise, remove it
#same thing from the end

listofR=[]
#print((np.shape(X)))
coef=0.4 #for first dataset it was 0.4, second 0.2 THIS NEEDS ADJUSTING
for col in range((np.shape(X))[0]):
    slice2=Z[:,col]
    R=abs(np.corrcoef(Y,slice2))[0,1]
    listofR.append(R)
    #print(R)
i=0
while listofR[i]<coef: #find uncorrelated data in the beginning 
    Z=np.delete(Z,0,1) #remove that row from Z
    X=np.delete(X,0,0) #remove the wavelength too
    i=i+1
i=(np.shape(X))[0]-1#same thing but from the end
while listofR[i]<coef:
    Z=np.delete(Z,-1,1)
    X=np.delete(X,-1,0)
    i=i-1

#step 4: remove the laser
#add a manual option
fileinforead=open(fileinfo)
pumpwl=0
for line in fileinforead:
    if line[0:15]=='Pump wavelength':
        pumpwl= int(line[-4:-1])
        print(pumpwl)
if pumpwl!=0:
    starti = np.searchsorted(X, (pumpwl - 20))#this can be adjusted
    endi = np.searchsorted(X, (pumpwl + 20))
    Z[:,starti:endi]=0
if pumpwl==0:
    print("did not find data")

600


In [273]:
%matplotlib widget
plt.pcolormesh(X,Y,Z, cmap='jet', shading='nearest')
#shading='nearest' makes the pixels centered on the coordinates
#plt.yscale('log')

plt.ylabel('Time(ps)')
plt.xlabel('Wavelength(nm)')
plt.ylim(-1,1)
plt.savefig("mygraph.png")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [274]:
#finds the important wavelengths for the gaussian
#n=10
n=int(input("how many points do you want the curve to be based on? "))

maxZ=np.amax(Z)
fiveper = maxZ*0.05
x=[0]#array of indices of even chunks
maxes=[] #temporary array of max values of each wl in the chunk
c=(X[-1]-X[0])/n #how big the increments are

xmaxes=[]#array of wls for which to base gaussian
for i in range(1,n+1):
    x.append(np.searchsorted(X,(X[0]+i*c))) #end index
    
    for wl in range(x[i-1],x[i]):
        maxes.append(np.amax(Z[:,wl]))#finds max of each of 50 wls, appends to maxes[]
    if np.amax(maxes)< fiveper:
        maxes=[] #need to clear maxes
        continue
    else:
        wl= maxes.index(np.amax(maxes))+x[i-1] #finds index of max(maxes), adds to start of chunk
        xmaxes.append(wl)
    maxes=[] #need to clear maxes 
print(xmaxes)


how many points do you want the curve to be based on? 15
[24, 44, 56, 90, 100, 144, 196, 200, 234, 248, 270, 297, 329, 365]


In [275]:
def curve3(t, y0,pw,t0,a1,k1,a2,k2,a3,k3):
    return y0+0.5*a1*(e**(-k1*(t-t0-(pw**2)*k1/2)))*(1+erf((t-t0-k1*(pw**2))/(1.41421356237*pw)))+0.5*(a2*e**(-k2*(t-t0-(pw**2)*k2/2)))*(1+erf((t-t0-k2*(pw**2))/(1.41421356237*pw)))+0.5*(a3*e**(-k3*(t-t0-(pw**2)*k3/2)))*(1+erf((t-t0-k3*(pw**2))/(1.41421356237*pw)))
def curve4(t, y0,pw,t0,a1,k1,a2,k2,a3,k3,a4,k4):#,a5,k5,a6,k6):
    return y0+0.5*a1*(e**(-k1*(t-t0-(pw**2)*k1/2)))*(1+erf((t-t0-k1*(pw**2))/(1.41421356237*pw)))+0.5*(a2*e**(-k2*(t-t0-(pw**2)*k2/2)))*(1+erf((t-t0-k2*(pw**2))/(1.41421356237*pw)))+0.5*(a3*e**(-k3*(t-t0-(pw**2)*k3/2)))*(1+erf((t-t0-k3*(pw**2))/(1.41421356237*pw)))+0.5*a4*(e**(-k4*(t-t0-(pw**2)*k4/2)))*(1+erf((t-t0-k4*(pw**2))/(1.41421356237*pw)))
def curve5(t, y0,pw,t0,a1,k1,a2,k2,a3,k3,a4,k4,a5,k5):
    return y0+0.5*a1*(e**(-k1*(t-t0-(pw**2)*k1/2)))*(1+erf((t-t0-k1*(pw**2))/(1.41421356237*pw)))+0.5*(a2*e**(-k2*(t-t0-(pw**2)*k2/2)))*(1+erf((t-t0-k2*(pw**2))/(1.41421356237*pw)))+0.5*(a3*e**(-k3*(t-t0-(pw**2)*k3/2)))*(1+erf((t-t0-k3*(pw**2))/(1.41421356237*pw)))+0.5*a4*(e**(-k4*(t-t0-(pw**2)*k4/2)))*(1+erf((t-t0-k4*(pw**2))/(1.41421356237*pw)))+0.5*a5*(e**(-k5*(t-t0-(pw**2)*k5/2)))*(1+erf((t-t0-k5*(pw**2))/(1.41421356237*pw)))
def curve6(t, y0,pw,t0,a1,k1,a2,k2,a3,k3,a4,k4,a5,k5,a6,k6):
    return y0+0.5*a1*(e**(-k1*(t-t0-(pw**2)*k1/2)))*(1+erf((t-t0-k1*(pw**2))/(1.41421356237*pw)))+0.5*(a2*e**(-k2*(t-t0-(pw**2)*k2/2)))*(1+erf((t-t0-k2*(pw**2))/(1.41421356237*pw)))+0.5*(a3*e**(-k3*(t-t0-(pw**2)*k3/2)))*(1+erf((t-t0-k3*(pw**2))/(1.41421356237*pw)))+0.5*a4*(e**(-k4*(t-t0-(pw**2)*k4/2)))*(1+erf((t-t0-k4*(pw**2))/(1.41421356237*pw)))+0.5*a5*(e**(-k5*(t-t0-(pw**2)*k5/2)))*(1+erf((t-t0-k5*(pw**2))/(1.41421356237*pw)))+0.5*pow(a6,(-k6*(t-t0-pow(pw,2)*k6/2)))*(1+erf((t-t0-k6*pow(pw,2))/(1.41421356237*pw)))

curve3model=Model(curve3)
curve4model=Model(curve4)
curve5model=Model(curve5)


def findT0(params,Y,Z,guess):#,pwguess):
    #-----The bad old way----
    #params, pc = curve_fit(curve1, Y,Z,maxfev=1000000, p0=[1e-6,0.1,guess,1e-3,1,1e-3,1e-2,1e-3,1e-3,1e-4,1e-3])
    #print(params[1], params[2], X[wl])
    ##pwguess[0]=params[1]
    #return params[2]
    
    #------the good new way-----
    #params['pw'].value= pwguess
    params['t0'].value= guess
    params['t0'].max= guess+0.2
    params['t0'].min= guess-0.2
    
    lm_output= curvemodel.fit(Z,params, t=Y, nan_policy='propagate')
    bestvals= lm_output.best_values
    return bestvals['t0']

In [276]:
#gaussian
leftguess=-0.4 #ADJUST -0.4,-0.2
curvemodel=curve3model
#print('parameter names: {}'.format(curvemodel.param_names))
#print('independent variables: {}'.format(curvemodel.independent_vars))

params = curvemodel.make_params(y0=1e-6,pw=0.1,t0=leftguess,a1=1e-3,k1=1,a2=1e-3,k2=1e-2,a3=1e-3,k3=1e-3)#,a4=1e-4,k4=1e-3)#,a5=1e-5,k5=1e-3)

params['k1'].min=0
params['k2'].min=0
params['k3'].min=0
#params['k4'].min=0
#params['k5'].min=0


#print(T0)
#pwguess=[0.1]
T0=[leftguess]
for wl in xmaxes:# range(0,np.shape(X)[0]): #
    #the indices around the pump wavelength that are all 0
    if starti<=wl<=endi: 
        T0.append(0)
        continue
    #interpolate the vertical slice over more data points
    f=interp1d(Y,Z[:,wl], kind="linear", fill_value="extrapolate")   
    Ybig=np.linspace(-100,2600,20000) #start,end,num of points
    Zbig=f(Ybig)
    
    T0.append(findT0(params,Y,Z[:,wl],T0[-1]))#,pwguess[0])) 
    print(T0[-1],X[wl])
print("done!")

-0.20000000002013957 435.201992885
-2.5927593405583593e-11 458.662530085
0.09966387606755939 472.765379125
0.26497783975097244 512.831492525
0.30952268498542956 524.646038725
0.4390565836624369 576.794176085
0.5450660040538772 638.768640325
0.5515776501907887 643.551380725
0.5794697833881733 684.293925485
0.7794697833354468 701.116689205
0.8289693918810536 727.607172125
0.631191662973867 760.209612185
0.8039376347690576 798.979964185
0.9803736762014079 842.765718025
done!


In [127]:
#----FOR TESTING-----
%matplotlib widget
fig=plt.figure()
wl=480
i=np.searchsorted(X,wl)
#params['t0'].value=
one_output= curvemodel.fit(Z[:,i],params, t=Y, nan_policy='propagate')
bestvals1= one_output.best_values
print(bestvals1['t0'])
curveplot=one_output.plot(fig=fig) #plots fit and resids. use plot_fit() or plot_residuals() for each
plt.xlim(-1,2)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0.6042640658970795


(-1.0, 2.0)

In [126]:
#---For testing but with slider----
%matplotlib widget
fig, ax = plt.subplots()


def slider(wl):
    #finds closest index to that t
    i=np.searchsorted(X,wl)
    #get horizontal slice
    plt.clf()
    
    one_output= curvemodel.fit(Z[:,i],params, t=Y, nan_policy='propagate')
    one_output.plot(fig=fig) #plots fit and resids. use plot_fit() or plot_residuals() for each
    plt.xlim(-1,2)
    
    
widgets.interact(slider, wl=(400,800,2))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=600, description='wl', max=800, min=400, step=2), Output()), _dom_classe…

<function __main__.slider(wl)>

In [277]:
def chirp_curve(maxwls, a, b, c,d):
    return a*pow(maxwls,3)+b*pow(maxwls,2)+c*maxwls+d

maxwls=[407] #ADJUST 407,435
for i in xmaxes: #converts list of indices to list of wls
    maxwls.append(X[i])
    
params1, pc = curve_fit(chirp_curve, maxwls, T0, maxfev=6000)
#params1, pc = curve_fit(chirp_curve, X, T0[:-1], maxfev=6000)
t0=chirp_curve(X, params1[0], params1[1], params1[2],params1[3])

%matplotlib widget
plt.pcolormesh(X,Y,Z, cmap='jet', shading='nearest')

plt.plot(maxwls,T0, color="yellow")
#plt.plot(X,T0[:-1], color="yellow")

plt.plot(X,t0, color="white")
plt.ylim(-1,1)
#plt.colorbar()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(-1.0, 1.0)

In [278]:
#CHIRP CORRECTOR-------
it0=[]
for wl in range(0,np.shape(X)[0]):
    it0.append(np.searchsorted(Y,t0[wl]))#gets the index of that wl's t0
    
#T0=float(y[0])#-0.6
iT0=np.searchsorted(Y,T0[0]) #index of first time zero
Znew=Z.copy()
Ynew=Y.copy()
for wl in range(0,np.shape(X)[0]):
    shift=0-t0[wl] #note, not all data is around zero, might have to change
    for t in range(iT0,np.shape(Y)[0]-1):#start changing from main T0
        Ynew[t]=Y[t]+shift
        Znew[t,wl]=Z[t,wl]
    f=interp1d(Ynew,Znew[:,wl], kind="linear", fill_value="extrapolate")       
    Znew[:,wl]=f(Y)

#plotter------
%matplotlib widget
fig, (ax1, ax2) = plt.subplots(1, 2)

ax1.pcolormesh(X,Y,Z, cmap='jet', shading='nearest') #before
ax1.axis(ymin=-1,ymax=2)
ax2.pcolormesh(X,Y,Znew, cmap='jet', shading='nearest')#manual curve
ax2.axis(ymin=-1,ymax=2)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(406.53809747500003, 844.593958375, -1.0, 2.0)

In [491]:
class SVD_data:
    def __init__(self, X,Y,Z):
        self.X = X
        self.Y = Y
        self.Z = Z
        self.u, self.s, self.vh = np.linalg.svd(Z) 
        self.rankdata ={}
    
    def get_rankdata(self,*list_of_ranks):
        for numRank in list_of_ranks:
            if not (numRank in self.rankdata.keys()): 
                self.rankdata[numRank]= Rank(numRank,self)
        return self.rankdata[numRank]
    
    def plot_s(self,numranks):
        x_axis=list(range(1, numranks+1))
        plt.scatter(x_axis,self.s[:numranks])
        plt.yscale('log')
        plt.title('s')
        
    
    def plot_u_and_vh(self,*args):
        working=True
        for numRank in args:
            if not (numRank in self.rankdata.keys()): 
                print(f"ERROR: rank={numRank} is not defined. Please call SVD_data.get_rankdata(rank) to define it.")
                working = False
        if not working: sys.exit();
        if (args):
            self.rankdata[args[0]].plot_u_and_vh()
        elif (not hasattr(self, 'goodrank')):
            print("ERROR: you have not specified the rank(int) you want to plot the u and vh for as an argument. This function can also be called without arguments if you set the preferred rank with SVD_Data.set_good_rank(ranknum) ")            
        else:
            self.goodrank.plot_u_and_vh()
            
        
    def set_good_rank(self,rank):
        self.goodrank=self.get_rankdata(rank) 
    
    def plot_ddA(self, *args):
        working=True
        for numRank in args:
            if not (numRank in self.rankdata.keys()): 
                print(f"ERROR: rank={numRank} is not defined. Please call SVD_data.get_rankdata(rank) to define it.")
                working = False
        if not working: sys.exit();
        if (args):
            self.rankdata[args[0]].plot_ddA()
        elif (not hasattr(self, 'goodrank')):
            print("ERROR: you have not specified the rank(int) you want to plot the ddA for as an argument. This function can also be called without arguments if you set the preferred rank with SVD_Data.set_good_rank(ranknum) ")            
        else:
            self.goodrank.plot_ddA()
            
    def plot_reconstr(self, *args):
        working=True
        for numRank in args:
            if not (numRank in self.rankdata.keys()): 
                print(f"ERROR: rank={numRank} is not defined. Please call SVD_data.get_rankdata(rank) to define it.")
                working = False
        if not working: sys.exit();
        if (args):
            self.rankdata[args[0]].plot_reconstr()
        elif (not hasattr(self, 'goodrank')):
            print("ERROR: you have not specified the rank(int) you want to plot the reconstruction for as an argument. This function can also be called without arguments if you set the preferred rank with SVD_Data.set_good_rank(ranknum) ")            
        else:
            self.goodrank.plot_reconstr()
            
class Rank:
    def __init__(r, rank,svd_data): #want to do things without all these other vars
        r.rank = rank
        
        #u, s, vh = np.linalg.svd(svd_data.Z)
        r.u_ranked= svd_data.u[:,:rank]
        r.vh_ranked = np.transpose(svd_data.vh[:rank:,])
        s_ranked=np.diag(svd_data.s[:rank])
        
        #creating reconstruced dataset
        step1=np.matmul(r.u_ranked,s_ranked)
        step2=np.transpose(r.vh_ranked)
        r.reconstructed=np.matmul(step1,step2)
        
        r.ddA=svd_data.Z-r.reconstructed
        
    def plot_u_and_vh(r):
        fig, (ax1, ax2) = plt.subplots(1, 2)

        ax1.plot(Y,r.u_ranked)
        ax1.set_xlabel('Time(ps)')
        ax1.set_ylabel('U')


        ax2.plot(X,r.vh_ranked)
        ax2.set_xlabel('Wavelength(nm)')
        ax2.set_ylabel('Vh')
        
    def plot_reconstr(r):
        plt.pcolormesh(X,Y,r.reconstructed, cmap='jet', shading='nearest')
        plt.colorbar()
        plt.ylabel('Time(ps)')
        plt.xlabel('Wavelength(nm)')
        plt.title(f'Reconstructed dataset from SVD for rank={r.rank}')
        #probably want to return this as an object...
        #plt.ylim(-0.1,2.5)
        
    def plot_ddA(r): #must deal with getting the X and Y
        plt.pcolormesh(X,Y,r.ddA, cmap='jet', shading='nearest')
        plt.colorbar()
        plt.ylabel('Time(ps)')
        plt.xlabel('Wavelength(nm)')
        plt.title(f'Delta (Delta A) for rank={r.rank}')
        #plt.ylim(-0.1,2.5)


In [503]:
%matplotlib widget

test_dataset=SVD_data(X,Y,Znew)

test_dataset.get_rankdata(1,2,3)
#test_dataset.set_good_rank(3) 
test_dataset.plot_u_and_vh(3)
#test_dataset.plot_s(10)

#rank_two=test_dataset.get_rankdata(4)
#test_dataset.plot_reconstr(2)
plt.ylim(-0.1,2.5)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [375]:
#testing how variables and objects in python work
from sys import getrefcount

print(getrefcount(s))
asdf=0
print(getrefcount(s))
asdf=s[3:5]
print(getrefcount(s))

#trying to make a template so that those two functions don't take up as much space 
#but i think that is a pointless endeavor
def plot_temp(self, func, *args):
        working=True
        for numRank in args:
            if not (numRank in self.rankdata.keys()): 
                print(f"ERROR: rank={numRank} is not defined. Please call SVD_data.get_rankdata(rank) to define it.")
                working = False
        if not working: sys.exit();
        if (args):
            self.rankdata[args[0]].func()
        elif (not hasattr(self, 'goodrank')):
            print("ERROR: you have not specified the rank(int) you want to plot the ddA for as an argument. This function can also be called without arguments if you set the preferred rank with SVD_Data.set_good_rank(ranknum) ")            
        else:
            self.goodrank.func()
            
def plot_reconstr(self,*args):
    self.plot_temp(Rank.plot_reconstr(), args)

3
2
3


In [303]:
#--finds SVD, plots first ten values of s on log scale----
%matplotlib widget
u,s,vh = np.linalg.svd(Znew)
#print("u= ", u,"s= ",s,"vh= ",vh)
print(np.shape(u)) #time
print(np.shape(vh)) #spectra
ten=list(range(1, 11))
plt.scatter(ten,s[:10])
plt.yscale('log')
plt.title('s')

#---plots u vs wl and vh vs time
fig, (ax1, ax2) = plt.subplots(1, 2)

rank =4
u_ranked= u[:,:rank]
vh_ranked = np.transpose(vh[:rank:,])

ax1.plot(Y,u_ranked)
#ax1.plot(Y,u_ranked[:,0], label="1")
#ax1.plot(Y,u_ranked[:,1], label="2")
#ax1.plot(Y,u_ranked[:,2], label="3")
#ax1.legend()
ax1.set_xlabel('Time(ps)')
ax1.set_ylabel('U')


ax2.plot(X,vh_ranked)
#ax2.plot(X,vh_ranked[:,0], label="1")
#ax2.plot(X,vh_ranked[:,1], label="2")
#ax2.plot(X,vh_ranked[:,2], label="3")
#ax2.legend()
ax2.set_xlabel('Wavelength(nm)')
ax2.set_ylabel('Vh')

#---Reconstructing the dataset from SVD---

s_ranked=np.diag(s[:rank])
#print(s_ranked)
step1=np.matmul(u_ranked,s_ranked)
#print(np.shape(step1))
step2=np.transpose(vh_ranked)
#print(np.shape(step2))
step3=np.matmul(step1,step2)
#print(np.shape(step3))
plt.pcolormesh(X,Y,step3, cmap='jet', shading='nearest')
plt.colorbar()
plt.ylabel('Time(ps)')
plt.xlabel('Wavelength(nm)')
plt.title('Reconstructed dataset from SVD')
#plt.ylim(-0.1,2.5)
#plt.xlim(406,450)

#---plotting the residual-----
%matplotlib widget
ddA= Znew-step3
print(np.shape(ddA))
plt.pcolormesh(X,Y,ddA, cmap='jet', shading='nearest')
plt.colorbar()
plt.ylabel('Time(ps)')
plt.xlabel('Wavelength(nm)')
plt.title('Delta (Delta A)')
#plt.ylim(-0.1,2.5)
#plt.xlim(406,450)


(455, 455)
(367, 367)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 's')

In [362]:
print(Znew.mean())# rank=1 mean=-2.1877917989063186e-06, rank=4 mean=5.802757866609229e-07
#can compare mean of residual... the lower the mean the closer it is to the dataset
#but what is the baseline for the noise? like what is a good number?
#could plot the means and see when there is a sharp decrease? 

#options: mean, corcoeff, autocorrelation,??


for col in range((np.shape(X))[0]):
    slice2=ddA[:,col]
    R=abs(np.corrcoef(Y,slice2))[0,1]
    #listofR.append(R)
    #print(R)
for row in range((np.shape(Y))[0]):
    slice3=ddA[row,:]
    R1=abs(np.corrcoef(X,slice3))[0,1]
    #listofR.append(R)
    #print(R1)

0.0004868824977166069


In [235]:
#---this grabs the fixed data from IGOR to compare the chirping

filename="P-1-3A-P-ex600nm-50uW-fs-chirped.csv"#22BPNODIPS chloroform 600 200mW PTA fixed.csv
rawdata = open(filename)

#step 1: split the data from the info at the end
#making it into new files: optional, can stick them into variables instead
fileinfo=filename[0:-4]+"-info"+".csv"
fileinfofile= open(fileinfo, "w") #'w' makes it write only so deletes prev version of it
justdata=filename[0:-4]+"-data"+".csv"
justdatafile=open(justdata, "w")

for line in rawdata:
    if len(line)<100: #if it is not data but text
        fileinfofile.write(line) 
    #step 2: remove the NaNs at the begining
    else: 
        nan=False
        for char in line:
            if char == 'n':
                nan=True
                break
        if nan:
            continue
        else:
            justdatafile.write(line)
fileinfofile.close() #its good practice to close files :)
rawdata.close()
justdatafile.close()
print((open(fileinfo)).read())
dataarray1 = (np.genfromtxt(justdata, delimiter=",",  filling_values=0)).T
print(np.shape(dataarray1))
A=dataarray1[0,1:].T #works
B=dataarray1[1:,0]
C=dataarray1[1:,1:]


(336, 1025)


In [236]:
#step 3: remove the noise at the begining and end
#read the begining, check if its noise, remove it
#same thing from the end

listofR=[]
#print((np.shape(X)))
coef=0.2 #for first dataset it was 0.4 THIS NEEDS ADJUSTING
for col in range((np.shape(A))[0]):
    slice2=C[:,col]
    R=abs(np.corrcoef(B,slice2))[0,1]
    listofR.append(R)
    print(R)
i=0
while listofR[i]<coef: #find uncorrelated data in the beginning 
    C=np.delete(C,0,1) #remove that row from Z
    A=np.delete(A,0,0) #remove the wavelength too
    i=i+1
i=(np.shape(A))[0]-1#same thing but from the end
while listofR[i]<coef:
    C=np.delete(C,-1,1)
    A=np.delete(A,-1,0)
    i=i-1

#step 4: remove the laser
#add a manual option
fileinforead=open(fileinfo)
pumpwl=600
for line in fileinforead:
    if line[0:15]=='Pump wavelength':
        pumpwl= int(line[-4:-1])
        print(pumpwl)
if pumpwl!=0:
    starti = np.searchsorted(A, (pumpwl - 10))#this can be adjusted
    endi = np.searchsorted(A, (pumpwl + 10))
    C[:,starti:endi]=0
if pumpwl==0:
    print("did not find data")
    
%matplotlib widget
fig, (ax1, ax2) = plt.subplots(1, 2)

ax1.pcolormesh(X,Y,Znew, cmap='jet', shading='nearest')
ax1.axis(ymin=-1,ymax=1)
ax2.pcolormesh(A,B,C, cmap='jet', shading='nearest')
ax2.axis(ymin=-1,ymax=1)

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
0.07668142247630812
nan
nan
nan
0.03979981402814614
0.02509232683505761
0.09459093066353935
0.020556707724703166
0.10941188720406787
0.005194714288896302
0.06333291385657529
0.022249692764339576
0.019676084301424573
0.022156612128117734
0.06673156638150797
0.000216566

0.15874394154113416
0.16609371762211503
0.15740669183231054
0.0772554669219778
0.06889090954620047
0.13667015294678614
0.1442220955573776
0.09848471271478326
0.15367481943058853
0.12010018725424719
0.1715741895553382
0.18700853930749944
0.12316828879414697
0.0905605083340238
0.14230366783466966
0.10427171668538597
0.11961048843117117
0.0012755190981025737
0.026124152249786148
0.06615349152917395
0.0546803792588849
0.0013525526220882753
0.04227359938440635
0.10473540857917536
0.012418649691006006
0.09859722208517115
0.019246771932621618
0.020058288011741053
0.01842225443924988
0.05130001165548126
0.019510576255005414
0.10631427612099531
0.0035448302036289133
0.004616246051245454
0.11579217457814862
0.06901061940511587
0.09428368668464626
0.016322713033731794
0.06475382348962937
0.0372628953501123
0.057191471297317636
0.01066462288502976
0.11847252908242843
0.007705598721640189
0.07234267572420311
0.03072266276725839
0.0693090261221074
0.06482852556796512
0.12834291240845971
0.0261517984

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(307.40142822250004, 912.0622253414999, -1.0, 1.0)

In [38]:
#thing to check how different i am from the properly fixed one
%matplotlib widget

D=Znew[:453,8:344]-C #datasets are different sizes, this centers them
for wl in range(0,np.shape(C)[1]): 
    for t in range(0,np.shape(C)[0]):
        D[t,wl]=D[t,wl]-C[t,wl]

plt.pcolormesh(X[:336],Y[:453],D, cmap='jet', shading='nearest')
plt.ylim(0,10)
plt.colorbar()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [130]:
#horizontal slice
%matplotlib widget
fig, ax = plt.subplots()


def slider(t):
    #finds closest index to that t
    i = np.searchsorted(Y, t)
    #get horizontal slice
    slice1=Znew[i,:]
    [l.remove() for l in ax.lines]
    ax.plot(X,slice1,color="black")
   
    
widgets.interact(slider, t=(-1,1,0.1))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=0.0, description='t', max=1.0, min=-1.0), Output()), _dom_classes=('wi…

<function __main__.slider(t)>

In [41]:
#VERTICAL SLICE----------
%matplotlib widget
fig, ax = plt.subplots()


def slider(wl):
    #finds closest index to that wl
    i = np.searchsorted(X, 512)
    print(i)
    slice2=Z[:,i];
    [l.remove() for l in ax.lines]
    ax.plot(Y,slice2,color="black")

#Slider used to select wavelength
widgets.interact(slider, wl=(400,700,3))

#need to give it a different amplitude sign
#do the fit with a few initial guesses
#giving it a bad value for pw? so need to find a better thing



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=550, description='wl', max=700, min=400, step=3), Output()), _dom_classe…

<function __main__.slider(wl)>

In [ ]:
#error function....
#define parameters of the chunk of plot we r looking at
#loop through all vertical slices and do the error function thing
#use those points as the curve
#stretch/interpolate
start=np.searchsorted(Y, 0.6)
end=np.searchsorted(Y, -0.6)
wl=np.searchsorted(X, 528)
error=erf(Z[start:end,wl])
print(error)


In [80]:
np.float128(1) + np.float128(2**-64) - np.float128(1)
#an answer of 0.0 indicates that the float128 type contains no more than 64 bits of precision

0.0

In [22]:
#Finds Gaussian convoluted with 3 exponentials at this wavelength
%matplotlib widget
plt.plot(Y,Z[:,248]) #blue wl=700
plt.plot(Ybig,Zbig) #orange
ExpGauss2=curve1(Ybig,params[0],params[1],params[2],params[3],params[4],params[5],params[6],params[7],params[8])#,params[9],params[10],params[11],params[12])
plt.plot(Ybig,ExpGauss2)#green
delta2=Zbig-ExpGauss2
plt.plot(Ybig,delta2)#red
#[-3.36189614e-05  1.97442342e+00 -5.70929342e+03  1.00000217e+00
# 1.52916609e+03  1.00000220e+00  1.49084208e+03  1.00000168e+00
#  1.46483839e+03]

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
#DONT need this :)
#making the curve 
x=[]
y=[]
for i in range(0,10):
    x.append(400+i*50)    
    y.append(input("Y value for wavelength "+str(x[i])+":"))
#x=[400,450,500,550,600,650,700,750,800,850]
#y=['-0.65', '-0.2', '0.1', '0.25', '0.45', '0.5', '0.55', '0.6', '0.6',"0.65"]


Y value for wavelength 400:-0.65
Y value for wavelength 450:-0.2


In [78]:
#plots individual wavelength curve fit but the old way, needs fixing
%matplotlib widget
i = np.searchsorted(X, 655) #the wavelength with the issue
#i=91
print(i)
slice2=Z[:,i];

#plots the slice
plt.plot(Y,slice2,color="black")
guess=0.27
params, pc = curve_fit(curve3, Y,Z[:,i],maxfev=100000, p0=[1e-6,0.1,guess,1e-3,1,1e-3,1e-2,1e-3,1e-3])

#plots the curve fit 
curv = curve3(Y,params[0], params[1], params[2],params[3],params[4], params[5], params[6],params[7], params[8])
plt.plot(Y, curv)
plt.xlim(-1,1)

#prints what the curv thinks t0 is
print("what the curve calculates for the variable t0: ", params[2], "ps")
print("this is t0 that is eyeballed",Y[g], "ps")



210


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

TypeError: curve1() missing 2 required positional arguments: 'a4' and 'k4'

In [88]:
#also dont think i need this
#CHIRP CORRECTOR 2.0??-------
Znewer=Z.copy()
Ynewer=Y.copy()

for wl in range(0,np.shape(X)[0]):
    shift=0-T[wl] #note, not all data is around zero, might have to change
    for t in range(0,np.shape(Y)[0]-1):#start changing from main T0
        Ynewer[t]=Y[t]+shift
        Znewer[t,wl]=Z[t,wl]
    f=interp1d(Ynewer,Znewer[:,wl], kind="linear", fill_value="extrapolate")       
    Znewer[:,wl]=f(Y)
print('done!')

done!


In [81]:
#saves everything
fixeddata =np.insert(Znew,0,Y, axis=1)
Xx= np.insert(X,0,0,axis=0)
Fixeddata= np.insert(fixeddata,0,Xx,axis=0)
np.savetxt(justdata, Fixeddata, delimiter=",")